<a href="https://colab.research.google.com/github/hamagami/preprocess/blob/main/04_06_%E3%82%A8%E3%83%B3%E3%82%B3%E3%83%BC%E3%83%89%E3%81%A8%E6%AD%A3%E8%A6%8F%E5%8C%96%E3%81%AE%E4%BE%8B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# エンコードと正規化

必要な基本モジュールと設定の読み込み

In [ ]:
import pandas as pd
pd.set_option('display.max_columns', 50)
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_style("whitegrid")
%matplotlib inline

[ UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/index.php)から[Automobile Data Set](https://archive.ics.uci.edu/ml/datasets/automobile)を取得する。このデータにはカラム名がついていないため，自分でカラム名を設定。ためしに最初の10レコードだけ出力させる

In [ ]:
headers = ["symboling", "normalized_losses", "make", "fuel_type", "aspiration",
           "num_doors", "body_style", "drive_wheels", "engine_location",
           "wheel_base", "length", "width", "height", "curb_weight",
           "engine_type", "num_cylinders", "engine_size", "fuel_system",
           "bore", "stroke", "compression_ratio", "horsepower", "peak_rpm",
           "city_mpg", "highway_mpg", "price"]

df = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/autos/imports-85.data",header=None, names=headers, na_values="?" )
df.head(10)

In [ ]:
df.dtypes #各カラムのデータの型を確認 

In [ ]:
df.isnull().any() #各カラムに NaN が存在しているかどうかを確認。NaNが入っているデータならTrueとなる

In [ ]:
df.isnull().sum()[df.isnull().sum() != 0] #NaNが入っているカラムごとにNaNの個数を確認

In [ ]:
df[df.isnull().any(axis=1)].head() #NaNがでてくる行だけセレクトして最初の方だけ出力

# データの統計的性質を把握する

In [ ]:
df.describe() #それぞれのカラムに対する統計量を確認 型が数値の場合のみでNaNは除外された統計量となる

各変数間の相関関係を図示してみる(先の例では Profiling で行ったが，ここでは直接グラフに出してみる）

In [ ]:
corr = df.corr()
# Generate a mask for the upper triangle
mask = np.zeros_like(corr, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(11, 9))

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, mask=mask, vmax=.8, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})

## ラベルエンコーダ
カテゴリカルデータのラベルをエンコードして数値に直す

In [ ]:
from sklearn.preprocessing import LabelEncoder # ラベルエンコーダを試す

In [ ]:
df["make"].unique() #カラム "make"のユニークな要素を確認

In [ ]:
le = LabelEncoder()
df["make_encode"] = le.fit_transform(df["make"]) #新たに make_encode というカラムをつくり，そこに makeの要素をエンコードした結果をいれる
df[["make", "make_encode"]].head(11) # make と make_encodeのペアを表示　メーカー（ラベル）ごとに異なる整数値（コード）が割り当てられる

## ラベルバイナライザ

In [ ]:
from sklearn.preprocessing import LabelBinarizer

In [ ]:
df["body_style"].unique() #車のスタイルに関するユニークな要素を抽出

In [ ]:
lb = LabelBinarizer()
lb_results = lb.fit_transform(df["body_style"]) #各要素をカラムとするマトリスクをつくる
pd.DataFrame(lb_results, columns=lb.classes_).head()

## ワンホットエンコーダー

In [ ]:
from sklearn.preprocessing import OneHotEncoder

In [ ]:
df[["make", "make_encode"]].head(11) #ラベルエンコーダの結果を再掲

In [ ]:
oh = OneHotEncoder()
oh.fit_transform(df["make_encode"].head(11).values.reshape(-1,1)).A #エンコード（整数）を各ビットで表現する　最初の11レコードに含まれるメーカーは3つなので3ビットにエンコードされる



---



---



## MinMax正規化の例

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
mms = MinMaxScaler()

In [ ]:
# ダミーデータをつくる
from sklearn.datasets import make_blobs
# 2次元，分散１のクラスタ１つ　100個の点をつくる
X, Y = make_blobs(random_state=8,
                  n_samples=100, 
                  n_features=2, 
                  cluster_std=1.,
                  centers=1)
# 描画
plt.figure(figsize=(7, 7))
plt.scatter(X[:, 0], X[:, 1], marker='o', c="r", s=25,label="original")
# それぞれの軸で　最大１　最小０にスケーリング
XX = mms.fit_transform(X)
plt.scatter(XX[:, 0], XX[:, 1], marker='o', c="b", s=25, label="Normarization")

plt.legend()

## 標準化

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
ss = StandardScaler()

In [ ]:
#　ダミーデータをつくる
from sklearn.datasets import make_blobs
# 2次元，分散2のクラスタ１つ　100個の点をつくる
X, Y = make_blobs(random_state=8,
                  n_samples=100, 
                  n_features=2, 
                  cluster_std=2.,
                  centers=1)

plt.figure(figsize=(7, 7))
plt.scatter(X[:, 0], X[:, 1], marker='o', c="r", s=25,label="original")
# 標準化（それぞれの軸に対して，平均０，分散１）
XX = ss.fit_transform(X)
plt.scatter(XX[:, 0], XX[:, 1], marker='o', c="g", s=25, label="standardization")

plt.legend()